## 1인 당 공원 면적(m^2 / 명)
1. 법적동으로 통일
2. 다른 지자체 동 이름 중복 -> 서울시만 추출
3. 구, 동 이름 중복 -> 구, 동 구분, 동 추출

In [1]:
import pandas as pd
import re

# 전국 공원 데이터 CSV 파일 읽기
park_file_path = '../data/park.csv'

# UnicodeDecodeError 해결 - 한글 인코딩 utf-8 euc-kr cp949
park_df = pd.read_csv(park_file_path, encoding='euc-kr')

# 전국 인구 데이터 CSV 파일 읽기
population_file_path = '../data/population.csv'
population_df = pd.read_csv(population_file_path, encoding='euc-kr') 

In [2]:
# 공원 데이터 주소가 비어 있거나 NaN인 경우 제거
park_df = park_df[park_df['소재지지번주소'].notna() & (park_df['소재지지번주소'] != '')]

# 공원 데이터 서울특별시만 추출
park_df = park_df[park_df['소재지지번주소'].str.contains('서울특별시')]

# 주소에서 공백 구분한 리스트를 프레임으로 변경 후 '구'를 추출
park_df['자치구'] = park_df['소재지지번주소'].str.split(' ', expand=True)[1]

# 주소에서 공백 구분한 리스트를 프레임으로 변경 후 '동'을 추출
park_df['법정동'] = park_df['소재지지번주소'].str.split(' ', expand=True)[2]

# 법정동, 자치구 별 공원면적 합 데이터 프레임으로
filtered_park_df = park_df.groupby(['법정동', '자치구'])['공원면적'].sum().reset_index()

In [3]:
# 인구 데이터 서울특별시만 추출
population_df = population_df[(population_df['시도명'] == '서울특별시')]

# 인구 데이터 법정동, 자치구, 인구 컬럼만 추출
filtered_population_df = pd.DataFrame({
    '법정동': population_df['읍면동명'],
    '자치구': population_df['시군구명'],
    '인구': population_df['계']
})

print(filtered_population_df)

     법정동  자치구     인구
0    청운동  종로구   2289
1    신교동  종로구   1340
2    궁정동  종로구    211
3    효자동  종로구    570
4    창성동  종로구    404
..   ...  ...    ...
450  둔촌동  강동구  24349
451  암사동  강동구  65775
452  성내동  강동구  65333
453  천호동  강동구  84811
454  강일동  강동구  34099

[455 rows x 3 columns]


In [4]:
# 동별 인구 데이터와 공원 면적 데이터 인구 데이터 기준 병합
merged_df = pd.merge(filtered_park_df, filtered_population_df, left_on=['법정동', '자치구'], right_on=['법정동', '자치구'], how='right')

# 1인 당 공원 면적 계산
merged_df['1인 당 공원면적(m^2/명)'] = merged_df['공원면적'] / merged_df['인구']

# 인구 당 공원 면적 값 내림차순 정렬
merged_df.sort_values('1인 당 공원면적(m^2/명)', ascending = False)

print(merged_df)

     법정동  자치구      공원면적     인구  1인 당 공원면적(m^2/명)
0    청운동  종로구       NaN   2289               NaN
1    신교동  종로구       NaN   1340               NaN
2    궁정동  종로구       NaN    211               NaN
3    효자동  종로구       NaN    570               NaN
4    창성동  종로구       NaN    404               NaN
..   ...  ...       ...    ...               ...
450  둔촌동  강동구  632733.0  24349         25.985995
451  암사동  강동구    1022.0  65775          0.015538
452  성내동  강동구       NaN  65333               NaN
453  천호동  강동구       NaN  84811               NaN
454  강일동  강동구  211206.1  34099          6.193909

[455 rows x 5 columns]


In [5]:
# 최종 결과 데이터 프레임 저장
output_file_path = '../data/park_result.csv'
merged_df.to_csv(output_file_path, encoding='utf-8')

# 병합 전 공원 데이터 임시 저장
#filtered_park_df.to_csv('../data/result.csv', encoding='utf-8')